<a href="https://colab.research.google.com/github/Buyan-Kirill/SVT/blob/main/Homework_n1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
import scipy.integrate as integrate
from numpy.polynomial.polynomial import Polynomial

a, b = -1., 1. # Левая и правая границы интегрирования
n = 3 # Максимальная степень полиномов в системе

# Весовая функция в скалярном произведении
class weight:
    p = Polynomial([1.])
    def __init__(polynomial):
        p = polynomial

# Проверка полиномов на ортонормированность
def check_ortonormal_polynoms(ort_polynoms, weight, a = -1., b = 1., eps = 10e-8):
    n = len(ort_polynoms)
    for i in range(n):
        for j in range(i + 1, n):
            p = (weight.p * ort_polynoms[i] * ort_polynoms[j]).integ()
            if abs(p(b) - p(a)) > eps:
                return False
        p = (weight.p * ort_polynoms[i] * ort_polynoms[i]).integ()
        if abs(p(b) - p(a) - 1) > eps:
            return False
    return True

# Проверка обнуления полинома максимальной степени на собственных значениях матрицы А
def check_eigenvalues(max_polynom, eigenvalues, eps = 10e-8):
    for i in eigenvalues:
        if abs(max_polynom(i)) > eps:
            return False
    return True

# Нахождение системы ортогональных полиномов через матрицу Грамма
def find_polynoms_Gramm_Shmidt(n, weight, a = -1., b = 1.):
    Gramm = [[(weight.p * Polynomial([0.] * (i + j) + [1.])).integ()(b) - (weight.p * Polynomial([0.] * (i + j) + [1.])).integ()(a) for i in range(n + 1)] for j in range(n + 1)]
    R = np.linalg.cholesky(Gramm)
    R = np.transpose(R)
    R = np.linalg.inv(R)
    R = np.transpose(R)
    ort_polynoms = []
    for i in range(n + 1):
        ort_polynoms.append(Polynomial(R[i]))
    return ort_polynoms

# Класс для рекурсивного нахождения системы ортогональных полиномов
class Recursive_method:
    alpha_arr = []
    beta_arr = []

    # Метод рекурсивного нахождения системы ортогональных полиномов
    def find_polynoms_recursively(self, n, weight, a = -1., b = 1.):
        ort_polynoms = []
        p = weight.p.integ()
        ort_polynoms.append(Polynomial([math.sqrt(1. / (p(b) - p(a)))]))
        p1 = (weight.p * ort_polynoms[0] * ort_polynoms[0] * Polynomial([0., 1.])).integ()
        p2 = (weight.p * ort_polynoms[0] * ort_polynoms[0]).integ()
        alpha = (p1(b) - p1(a)) / (p2(b) - p2(a))
        self.alpha_arr.append(alpha)
        p = Polynomial([0., 1.]) * ort_polynoms[0] - alpha * ort_polynoms[0]
        p1 = (weight.p * p * p).integ()
        beta = (math.sqrt(p1(b) - p1(a)))
        self.beta_arr.append(beta)
        p = p / beta
        ort_polynoms.append(p)
        for i in range(1, n):
            p1 = (weight.p * ort_polynoms[i] * ort_polynoms[i] * Polynomial([0., 1.])).integ()
            p2 = (weight.p * ort_polynoms[i] * ort_polynoms[i]).integ()
            alpha = (p1(b) - p1(a)) / (p2(b) - p2(a))
            self.alpha_arr.append(alpha)
            p = Polynomial([0., 1.]) * ort_polynoms[i] - alpha * ort_polynoms[i] - self.beta_arr[i - 1] * ort_polynoms[i - 1]
            p1 = (weight.p * p * p).integ()
            beta = (math.sqrt(p1(b) - p1(a)))
            self.beta_arr.append(beta)
            p = p / beta
            ort_polynoms.append(p)
        return ort_polynoms

    # Нахождение собственных значений матрицы А
    def find_eigenvalues(self):
        n = len(self.alpha_arr)
        A = np.zeros((n, n))
        for i in range(len(self.alpha_arr)):
            A[i, i] = self.alpha_arr[i]
            if i > 0:
                A[i, i - 1] = self.beta_arr[i - 1]
            if i < n - 1:
                A[i, i + 1] = self.beta_arr[i]
        eigenvalues, eigenvectors = np.linalg.eig(A)
        return eigenvalues

ort_polynoms1 = find_polynoms_Gramm_Shmidt(n, weight, a, b)
print("Проверка первого метода: ", check_ortonormal_polynoms(ort_polynoms1, weight, a, b))
A = Recursive_method()
ort_polynoms2 = A.find_polynoms_recursively(n, weight, a, b)
print("Проверка второго метода: ", check_ortonormal_polynoms(ort_polynoms2, weight, a, b))
eigenvalues = A.find_eigenvalues()
print("Проверка собственных значений: ", check_eigenvalues(ort_polynoms2[n], eigenvalues))

Проверка первого метода:  True
Проверка второго метода:  True
Проверка собственных значений:  True
